[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

**_CHANGE: replace `langchain-openai` by `langchain-ollama`_**

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain-ollama langchain_core langchain_community

In [7]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-do for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2")

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [3]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages
llm.invoke(messages)

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-12-31T09:30:19.654237Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2045033958, 'load_duration': 826769583, 'prompt_eval_count': 27, 'prompt_eval_duration': 932000000, 'eval_count': 25, 'eval_duration': 276000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3ae1feb8-746e-4701-8436-210d08c09aad-0', usage_metadata={'input_tokens': 27, 'output_tokens': 25, 'total_tokens': 52})

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [4]:
llm.invoke("hello world")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-12-31T09:30:22.012135Z', 'done': True, 'done_reason': 'stop', 'total_duration': 466313250, 'load_duration': 30653542, 'prompt_eval_count': 27, 'prompt_eval_duration': 149000000, 'eval_count': 26, 'eval_duration': 285000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a31fb465-72a5-4d78-b581-fb333eeb8b16-0', usage_metadata={'input_tokens': 27, 'output_tokens': 26, 'total_tokens': 53})

In [5]:
llm.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-12-31T09:30:25.27018Z', 'done': True, 'done_reason': 'stop', 'total_duration': 271496333, 'load_duration': 35386250, 'prompt_eval_count': 27, 'prompt_eval_duration': 131000000, 'eval_count': 10, 'eval_duration': 104000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-a7515f79-46ea-44f9-9db0-8258f83c2868-0', usage_metadata={'input_tokens': 27, 'output_tokens': 10, 'total_tokens': 37})

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [8]:
_set_env("TAVILY_API_KEY")

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangChain?")

In [12]:
search_docs

[{'url': 'https://www.geeksforgeeks.org/introduction-to-langchain/',
  'content': 'Data Structures and Algorithms\nML & Data Science\nWeb Development\nLanguages\nInterview Corner\nCS Subjects\nJobs\nPractice\nContests\nIntroduction to Langchain\nLangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). LangChain follows a general pipeline where a user asks a question to the language model where the vector representation of the question is used to do a similarity search in the vector database and the relevant information is fetched from the vector database and the response is later fed to the language model. LangChain Key Concepts:\nThe main properties of LangChain Framework are :\nSetting up the environment\nInstallation of langchain is very simple and similar as you install other libraries using the pip command.\n Next, we create a .env file and store our API key in it as follows:\nPython\nNow, I am creating a new file 